# Building a Random Forest Classifier with Spark in Batch Mode

In this notebook, you'll learn the basics of working with Spark in batch mode to build a random forest classifier. Before digging in, make sure to read the background context in the related reading in the curriculum!

##  Import dependencies

First off, we need to import the tools we'll need from PySpark. The imports below allow us to connect to the Spark server, load our data, clean it, and prepare, execute, and evaluate a model.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col

## Set our constants

Next, we create a set of constants that we can refer to throughout the notebook. These are values that the rest of our code needs to run, but that we might need to change at some point (for instance, if the location of our data changes). 

In [2]:
CSV_PATH = "/home/ds/notebooks/datasets/UCI_HAR/allData.csv"
CSV_ACTIVITY_LABEL_PATH = "/home/ds/notebooks/datasets/UCI_HAR/activity_labels.csv"
APP_NAME = "UCI HAR Random Forest Example"
SPARK_URL = "local[*]"
RANDOM_SEED = 141107
TRAINING_DATA_RATIO = 0.8
RF_NUM_TREES = 10
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

## Connect to the server and load data

Now we're ready to connect to the Spark server. We do that (relying on the constants set above) and then load our labels (loaded into `activity_labels`) and activity data (loaded into `df`). 

In [3]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
activity_labels = spark.read.options(inferschema = "true").csv(CSV_ACTIVITY_LABEL_PATH)
df = spark.read.options(inferschema = "true").csv(CSV_PATH)

## Validate the data

If our data has been properly cleaned and prepared, it will meet the following criteria, which we'll verify in just a moment:

* The dataframe shape should be 10,299 rows by 562 columns
* All feature columns should be doubles. Note than one of the columns is for our label and it will not be double.
* There should be no nulls. This point is crucial because Spark will fail to build our vector variables for our classifier if there are any null values.

Let's confirm these points.

In [4]:
# Confirm the dataframe shape is 10,299 rows by 562 columns
print(f"Dataset shape is {df.count():d} rows by {len(df.columns):d} columns.")

Dataset shape is 10299 rows by 562 columns.


In [5]:
# Confirm that all feature columns are doubles via a list comprehension
# We're expecting 561 of 562 here, accounting for the labels column
double_cols = [col[0] for col in df.dtypes if col[1] == 'double']
print(f"{len(double_cols):d} columns out of {len(df.columns):d} total are type double.")

561 columns out of 562 total are type double.


In [6]:
# Confirm there are no null values. We use the dataframe select method to build a 
# list that is then converted to a Python dict. This way it's easy to sum up the nulls.
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
                         for c in df.columns]).toPandas().to_dict(orient='records')

print(f"There are {sum(null_counts[0].values()):d} null values in the dataset.")

There are 0 null values in the dataset.


## Set up and run our classifier in Spark

After confirming our data is clean, we're ready to reshape the data and run the random forest model.

In Spark, we manipulate the data to work in a Spark pipeline, define each of the steps in the pipeline, chain them together, and finally run the pipeline.

Apache Spark classifiers expect 2 columns of input:

1. __labels__: an indexed set of numeric variables that represent the classification from the set of features we provide.
2. __features__: an indexed, vector variable that contains all of the feature values in each row. 

In order to do this, we need to create these 2 columns from our dataset - the data is there, but not yet in a format we can use in the classifier.

To create the indexed labels column, we'll create a column called `indexedLabel` using the `StringIndexer` method. We use the column `_c0` as the source for our label index since that contains our labels. The column contains only one value per index.
    
To create the indexed features column, we'll need to do two things. First, we'll create the vector of features using the `VectorAssembler` method. To create this vector, we'll need to use all 561 numeric columns from our data frame. The vector assembler will create a new column called `features`, and each row of this column will contain a 561-element vector that is built from the 561 features in the dataset.

Finally, we'll complete the data preparation by creating an indexed vector from the `features` column. We'll call this vector `indexedFeatures`.
    
Since the classifier expects indexed labels and an indexed vector column of data, we'll use the `indexedLabel` and `indexedFeatures` as inputs to our random forest classifier.

In [7]:
# Generate our feature vector.
# Note that we're doing the work on the `df` object - we don't create new dataframes, 
# just add columns to the one we already are using.

# the transform method creates the column.

df = VectorAssembler(inputCols=double_cols, outputCol="features").transform(df)

Let's confirm that the features are there. It's easy to do this in Apache Spark using the `select` and `show` methods on the dataframe.  

In [8]:
df.select("_c0","_c1","_c2", "features").show(5)

+---+-----+-------+--------------------+
|_c0|  _c1|    _c2|            features|
+---+-----+-------+--------------------+
|  5|0.289|-0.0203|[0.289,-0.0203,-0...|
|  5|0.278|-0.0164|[0.278,-0.0164,-0...|
|  5| 0.28|-0.0195|[0.28,-0.0195,-0....|
|  5|0.279|-0.0262|[0.279,-0.0262,-0...|
|  5|0.277|-0.0166|[0.277,-0.0166,-0...|
+---+-----+-------+--------------------+
only showing top 5 rows



Now we're ready to build the indexers, split our data for training and testing, define our model, and finally chain everything together into a pipeline.

__It's important to note that when we execute this cell, we're not actually running our model. At this point, we're only defining its parameters__.

In [9]:
# Build the training indexers / split data / classifier
# first we'll generate a labelIndexer
labelIndexer = StringIndexer(inputCol="_c0", outputCol="indexedLabel").fit(df)

# now generate the indexed feature vector
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)
    
# Split the data into training and validation sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=RF_NUM_TREES)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

This next cell runs the pipeline, delivering a trained model at the end of the process.

In [11]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

It is now easy to test our model and make predictions simply by using the model's `transform` method on the `testData` dataset.

In [12]:
# Make predictions.
predictions = model.transform(testData)

## Evaluate the model

Now we can use the MulticlassClassificationEvaluator to test the model's accuracy.

In [13]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.0870425
Accuracy = 0.912957


In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionTrainingSummary
# Same but with LOGS
df_lr = df["_c0", "features"]
df_lr = df_lr.selectExpr("_c0 as label", "features as features")

#labelIndexer = StringIndexer(inputCol="_c0", outputCol="indexedLabel").fit(df_lr)
#featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)
(trainingData, testData) = df_lr.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lrModel = lr.fit(trainingData)

lrt = LogisticRegressionTrainingSummary(lrModel)
# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

lrModel.hasSummary
#lrModel.summary

#model = pipeline.fit(trainingData)
#predictions = model.transform(testData)
#df_lr.show()

Coefficients: 
7 X 561 CSRMatrix
(1,16) 0.0103
(2,281) 0.0105
(3,302) 0.0111
(3,310) 0.0111
(3,314) 0.0105
(3,381) 0.01
(3,508) 0.0149
(4,40) -0.2597
(5,47) -0.0001
(6,49) -0.2614
(6,52) -0.2631
(6,56) -0.1984
(6,60) -0.0039
(6,308) -0.0024
(6,333) -0.0151
(6,334) -0.017
..
..
Intercept: [-6.186717912730653,1.0408249437041404,0.9408860964524167,0.8392777973685462,1.0757789934654056,1.1409107751570735,1.1490393065830717]


False

In [30]:
df_lr.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    5|[0.289,-0.0203,-0...|
|    5|[0.278,-0.0164,-0...|
|    5|[0.28,-0.0195,-0....|
|    5|[0.279,-0.0262,-0...|
|    5|[0.277,-0.0166,-0...|
|    5|[0.277,-0.0101,-0...|
|    5|[0.279,-0.0196,-0...|
|    5|[0.277,-0.0305,-0...|
|    5|[0.277,-0.0218,-0...|
|    5|[0.281,-0.00996,-...|
|    5|[0.277,-0.0127,-0...|
|    5|[0.276,-0.0214,-0...|
|    5|[0.278,-0.0204,-0...|
|    5|[0.277,-0.0147,-0...|
|    5|[0.298,0.0271,-0....|
|    5|[0.279,-0.023,-0....|
|    5|[0.279,-0.0148,-0...|
|    5|[0.28,-0.0139,-0....|
|    5|[0.278,-0.0182,-0...|
|    5|[0.276,-0.017,-0....|
+-----+--------------------+
only showing top 20 rows



## Next Steps

We've seen how to prepare data and build a classifier in Spark. You might want to play around with this notebook and learn more about how Spark works. Here are some ideas:

- Look at the set of labels, and see if there are any features that would make sense to combine. Spark allows you to map values into a new column.
- Identify the most important features among the 561 source features (using PCA or something similar), then reduce the feature set and see if the model performs better.
- Modify the settings of the random forest to see if the performance improves.
- Use Spark's tools to find other techniques to evaluate the performance of your model. See if you can figure out how to generate an ROC plot, find the AUC value, or plot a confusion matrix.

In [65]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tr

In [66]:
df.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[0.289,-0.0203,-0...|
|[0.278,-0.0164,-0...|
|[0.28,-0.0195,-0....|
|[0.279,-0.0262,-0...|
|[0.277,-0.0166,-0...|
+--------------------+
only showing top 5 rows



In [67]:
activity_labels.show()

+---+------------------+
|_c0|               _c1|
+---+------------------+
|  1|           WALKING|
|  2|  WALKING_UPSTAIRS|
|  3|WALKING_DOWNSTAIRS|
|  4|           SITTING|
|  5|          STANDING|
|  6|            LAYING|
+---+------------------+



In [68]:
from pyspark.ml.linalg import Vectors
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),), 
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df2 = spark.createDataFrame(data,["features"])

In [69]:
df2.show()

+--------------------+
|            features|
+--------------------+
| (5,[1,3],[1.0,7.0])|
|[2.0,0.0,3.0,4.0,...|
|[4.0,0.0,0.0,6.0,...|
+--------------------+



In [70]:
from pyspark.ml.feature import PCA
pca = PCA(k=300, inputCol="features", outputCol="pca_features")
model = pca.fit(df)

In [71]:
result = model.transform(df).select("pca_features")
result.count()

10299

In [72]:
df.count()

10299

In [85]:
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(numTopFeatures=100, featuresCol="features",
                        outputCol="selectedFeatures", labelCol="_c0")
result = selector.fit(df).transform(df)

In [90]:
labelIndexer = StringIndexer(inputCol="_c0", outputCol="indexedLabel").fit(result)

# now generate the indexed feature vector
featureIndexer = VectorIndexer(inputCol="selectedFeatures", outputCol="indexedFeatures", maxCategories=4).fit(result)
    
# Split the data into training and validation sets (30% held out for testing)
(trainingData, testData) = result.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=RF_NUM_TREES)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [91]:
model = pipeline.fit(trainingData)
predictions = model.transform(testData)

In [92]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.0860421
Accuracy = 0.913958


In [108]:
labelIndexer = StringIndexer(inputCol="_c0", outputCol="indexedLabel").fit(result)
featureIndexer = VectorIndexer(inputCol="selectedFeatures", outputCol="indexedFeatures", maxCategories=4).fit(result)
(trainingData, testData) = result.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=20)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

model = pipeline.fit(trainingData)
predictions = model.transform(testData)

In [109]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.0901034
Accuracy = 0.909897
